In [70]:
import numpy as np

from scipy.optimize import fmin_tnc

In [71]:
np.random.seed(123)

In [72]:
N = 20  # Num runs
n = 100  # Num ratings per run
p = 0.4  # True relevance for p_k s.t. k=1

# True lambda distribution parameters
b = 0.95
a = 0.1

In [73]:
# True lambda distribution: lambda_i = a + (1-a)*b**(i-1)
lambdas = a + (1 - a)*b**np.arange(n)

In [74]:
# Generate the data by iterating over Equation 4
R = np.zeros((N,n))  # Keep track of individual ratings
M = np.zeros((N,n))  # Keep track of sample mean
ri = np.random.rand(N) < p  # First rating sampled from Bernoulli(p)
R[:,0] = ri
M[:,0] = ri
m = ri

# Iterate over timesteps (all runs in parallel)
for i in range(1,n):
    pri = lambdas[i]*p + (1 - lambdas[i])*m  # Calculate P(r_n=1|\bar{p}_{n-1})
    ri = np.random.rand(N) < pri  # Sample new rating
    m = (m*i + ri)/(i + 1)  # Update sample mean
    R[:,i] = ri  # Record the new rating
    M[:,i] = m  # Record the new sample mean

In [95]:
def f_l(x0, r, m_shifted, p):
    """Calculate the negative loglikehood of the data and its gradient wrt. a and b for a bandwagon process in 
    a form minimizable by Truncated Newton.
    
    Args:
        x0 (tuple): Initial guess for a and b.
        r (np.array): Observed ratings at each timestep w/ shape (N, n).
        m_shifted (np.array): Observed sample means before a given timestep w/ shape (N, n).
        p (float): Estimated relevance for k=1.
    Returns:
        L (float): Negative log likelihood (loss).
        gradient (tuple): Partial derivatives of L wrt. a and b.
    """
    a, b = [np.clip(np.array([x]), 1e-8, 1-1e-8)[:,None] for x in x0]  # a and b bounded away from 0 and 1.
    l = a + (1 - a)*b**np.arange(n)  # Current estimate of lambdas
    denom = l*p + (1 - l)*m_shifted
    L = -np.sum(r*np.log(denom) + (1 - r)*np.log(1 - denom))  # Negative loglikelihood
    dL_da = -np.sum((p - m_shifted)*(1 - b**np.arange(n))*(r/denom - (1 - r)/(1 - denom)))
    dL_db = -np.sum((p - m_shifted)*(1 - a)*(b**np.arange(-1, n - 1))*np.arange(n)*(r/denom - (1 - r)/(1 - denom)))
    gradient = (dL_da, dL_db)
    
    return L, gradient

In [100]:
x0 = (0.5, 0.5)  # Initial guess for a and b
m_shifted = np.concatenate((np.ones((N,1)), M[:,:-1]), axis=-1)  # Sample mean as it was prior to the rating at the same index
estimated_relevance = R.mean()  # Estimated relevance (\hat{p}^k in Section 5.2)
bounds = [(1e-8,1-1e-8)]*2  # Assume a and b bounded away from 0 and 1.

In [99]:
a_hat, b_hat = fmin_tnc(f_l2, x0, args=(R, m_shifted, estimated_relevance), bounds=bounds, maxfun=10000)[0]
print(a_hat, b_hat)

0.3305871826186462 0.9185348612606968


  NIT   NF   F                       GTG
    0    1  1.304781967901043E+03   3.43543368E+01
    1    3  1.304692920146605E+03   5.41365045E+00
    2   10  1.303287280137861E+03   1.22370858E+02
    3   14  1.303275671965722E+03   1.45579809E+02
    4   16  1.302774767550221E+03   7.62630166E-01
    5   18  1.302771222212182E+03   6.38465500E-01
    6   20  1.302770787933406E+03   1.78328473E-04
tnc: fscale = 74.8843
    7   22  1.302770787797092E+03   3.96516707E-05
    8   24  1.302770787615172E+03   7.05931524E-11
tnc: fscale = 119020
tnc: |fn-fn-1] = 0 -> convergence
    9   26  1.302770787615172E+03   3.94775600E-10
tnc: Converged (|f_n-f_(n-1)| ~= 0)
